## Contexto

El discurso de odio es cualquier expresión que promueva o incite a la discriminación, la hostilidad o la violencia hacia una persona o grupo de personas en una relación asimétrica de poder, tal como la raza, la etnia, el género, la orientación sexual, la religión, la nacionalidad, una discapacidad u otra característica similar.

En cambio, la incivilidad se refiere a cualquier comportamiento o actitud que rompe las normas de respeto, cortesía y consideración en la interacción entre personas. Esta puede manifestarse de diversas formas, tal como insultos, ataques personales, sarcasmo, desprecio, entre otras.

En esta tarea tendrán a su disposición un dataset de textos con las etiquetas `odio`, `incivilidad` o `normal`. La mayor parte de los datos se encuentra en español de Chile. Con estos datos, deberán entrenar un modelo que sea capaz de predecir la etiqueta de un texto dado.

El corpus para esta tarea se compone de 3 datasets:  
- [Multilingual Resources for Offensive Language Detection de Arango et al. (2022)](https://aclanthology.org/2022.woah-1.pdf#page=136)
- [Dataton UTFSM No To Hate (2022)](http://dataton.inf.utfsm.cl/)
- Datos generados usando la [API de GPT3 (modelo DaVinci 03)](https://platform.openai.com/docs/models/gpt-3).

Agradecimientos a los autores por compartir los datos y a David Miranda, Fabián Diaz, Santiago Maass y Jorge Ortiz por revisar y reetiquetar los datos en el contexto del curso "Taller de Desarrollo de Proyectos de IA" (CC6409), Departamento de Ciencias de la Computación, Universidad de Chile.

Los datos solo pueden ser usados con fines de investigación y docencia. Está prohibida la difusión externa.


## Tarea a resolver

Para esta tarea 2, buscaremos desarrollar un *benchmark* sobre una tarea de clasificación de NLP. Un benchmark es básicamente utilizar diferentes técnicas para resolver una misma tarea específica, en este caso seguiremos buscando alternativas para resolver el problema de clasificación de la tarea 1. Particularmente, se le pide:

- Implementar una arquitectura en RNN utilizando PyTorch.
- Utilizar transformers para revolver el problema de clasificación, en especifico utilizar BETO.
- Utilizar algún LLM utilizando Zero y Few short learning para resolver el problema de clasificación.


### Cargar el dataset


En esta sección, cargaremos el dataset desde el repositorio del módulo. Para ello ejecute las siguientes líneas:

In [ ]:
import pandas as pd

In [ ]:
# Dataset.
dataset_df = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")


### Analizar los datos

En esta sección analizaremos el balance de los datos. Para ello se imprime la cantidad de tweets de cada dataset agrupados por la intensidad de sentimiento.

In [ ]:
dataset_df.sample(20)

In [ ]:
dataset_df["clase"].value_counts()

In [ ]:
target_classes = list(dataset_df['clase'].unique())
target_classes

### Instalar librerias

Puede usar esta celda para instalar las librerías que estime necesario.

In [ ]:
%%capture



### Importar librerías

En esta sección, importamos la liberías necesarias para el correcto desarrollo de esta tarea. Puede utilizar otras librerías que no se en encuentran aquí, pero debe citar su fuente.

In [ ]:
'''
!pip uninstall torch torchvision torchaudio

!pip install torch==2.3.0 --index-url https://download.pytorch.org/whl/cpu
!pip uninstall scipy
!pip install scipy==1.11.4
!pip install torchtext
!pip install scikit-plot
'''

In [1]:
import nltk
import numpy as np

from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

# importe aquí sus clasificadores

import matplotlib.pyplot as plt

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

# Pytorch imports
import torch
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

from torch import nn
from torch.nn import functional as F

from tqdm import tqdm
from sklearn.metrics import accuracy_score
import gc

from torch.optim import Adam


OSError: Could not load this library: /home/mbustamc/Documentos/diplomados/dia_2025/cc66q_lenguaje-natural/laboratorios/.venv/lib/python3.11/site-packages/torchtext/lib/libtorchtext.so

### Crear un clasificador basado en RNN

En esta parte de le pide definir un clasificador utilizando `PyTorch` con alguna arquitectura en Redes Recurrentes. Para ello debe realizar todos los pasos vistos en el tutorial 5, por lo que se recomienda revisarlo. Importante, no puede replicar ningún ejemplo de los del tutorial 5, debe proponer sus propias arquitecturas. Se le recomienda leer como utilizar variaciones de la RNN, como la LSTM o GRU en `Pytorch`.

Para completa esta parte deberá replicar el flujo de trabajo de como utilizar `PyTorch`. Esta esctrictamente prohibido utilizar variaciones que resuelvan directamente este problema, como `PyTorch Lightning` o `TensorFlow`. Los pasos a completar son los siguientes:

#### Cargar el dataset.

In [177]:
from sklearn.model_selection import train_test_split

def split_df(df, test_size = 0.2, seed: int = 42):
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=seed, stratify=df['clase'])
    return train_df, test_df


def load_df(df):
   for _, row in df.iterrows():
     yield row['texto'], row['clase']


train_df, test_df = split_df(dataset_df)

train_dataset = load_df(train_df)
test_dataset = load_df(test_df)

#### Definir el vocabulario.

In [178]:
import re

def tokenizer_es(text: str):
    return re.findall(r"\b[\wáéíóúüñÁÉÍÓÚÜÑ]+\b", str(text).lower())

def normalize(text):
    text = re.sub(r"http\S+","<URL>", text)
    text = re.sub(r"@\w+","<USER>", text)
    text = re.sub(r"#(\w+)","\\1", text)     # quita # pero deja la palabra
    return text

def tokenizer_es_norm(t):
    return tokenizer_es(normalize(t))


tokenizer = get_tokenizer("basic_english")

def generate_tokens(example: iter):
    for text, _ in example:
        yield tokenizer_es_norm(text)



def build_vocab(example: iter, min_freq=1):
    vocab = build_vocab_from_iterator(
        generate_tokens(example),
        min_freq=min_freq,
        specials=["<PAD>", "<UNK>"]   # <--- añade PAD primero
    )
    vocab.set_default_index(vocab["<UNK>"])
    return vocab


vocab = build_vocab_from_iterator(
    build_vocabulary([train_dataset, test_dataset]), 
    min_freq=1, 
    specials=["<PAD>","<UNK>"]
)
pad_idx = vocab["<PAD>"]
vocab.set_default_index(vocab["<UNK>"])


print(f"Tamaño del vocabulario: {len(vocab)}")
print(list(vocab.get_stoi().items())[:50])

Tamaño del vocabulario: 28258
[('𝗙𝗜𝗡𝗔𝗟𝗜𝗭𝗔', 28256), ('𝗘𝗹', 28255), ('𝗗𝗘', 28254), ('𝗖𝗨𝗥𝗦𝗢', 28252), ('𝐌𝐚𝐠𝐝𝐚𝐥𝐞𝐧𝐚', 28249), ('𝐃𝐢𝐠𝐢𝐭𝐚𝐥', 28248), ('𝐁𝐢𝐛𝐥𝐢𝐨𝐭𝐞𝐜𝐚', 28247), ('방탄소년단', 28244), ('útero', 28243), ('úsalas', 28242), ('ópera', 28240), ('óleo', 28239), ('ñuke', 28238), ('ñora', 28236), ('ñoquis', 28235), ('ñino', 28233), ('ñero', 28232), ('ñato', 28231), ('ña', 28229), ('íntimas', 28228), ('íntimamente', 28227), ('éso', 28221), ('çal', 28217), ('árboles', 28216), ('ángulo', 28213), ('ángelo', 28212), ('ángel', 28210), ('ámense', 28209), ('½', 28205), ('zurich', 28202), ('zurdita', 28199), ('zurdis', 28198), ('zurdaje', 28196), ('zopiza', 28188), ('zoomjakoll', 28187), ('zoo', 28184), ('zona0', 28182), ('zionista', 28180), ('zikes', 28179), ('zendal', 28176), ('zekhem', 28174), ('zarro', 28173), ('zarpes', 28172), ('zares', 28169), ('zanjas', 28165), ('zancos', 28164), ('zamorano', 28162), ('zambrano', 28161), ('zalaquett', 28160), ('zacatecas', 28159)]


#### Definir la red recurrente.

Recuerde que debe difirnir los hyperparametros que estime conveniente.

In [165]:
embed_len = 50
hidden_dim = 50
n_layers=1


class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_len, hidden_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_len)
        self.rnn = nn.GRU(embed_len, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_classes)
        
    def forward(self, X):
        # X: (batch, seq_len) con padding = 0
        emb = self.embedding(X)  # (batch, seq_len, embed_len)
        
        # longitudes reales (contar tokens != 0)
        lengths = (X != 0).sum(dim=1)

        packed = torch.nn.utils.rnn.pack_padded_sequence(
            emb, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        _, hidden = self.rnn(packed)               # hidden: (num_layers, batch, hidden_dim)
        logits = self.linear(hidden[-1])           # (batch, num_classes)
        return logits


#### Funciones de entrenamiento y evaluación.

Para esta parte, puede utilizar las funciones vista en el tutorial directamente. Pero es su reponsabilidad ajustarlas a su código.

In [166]:
def CalcValLossAndAccuracy(model, loss_fn, val_loader):
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())

            Y_shuffled.append(Y)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f}".format(torch.tensor(losses).mean()))
        print("Valid Acc  : {:.3f}".format(accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())))


def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        losses = []
        for X, Y in tqdm(train_loader):
            Y_preds = model(X)

            loss = loss_fn(Y_preds, Y)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(losses).mean()))
        CalcValLossAndAccuracy(model, loss_fn, val_loader)

def MakePredictions(model, loader):
    Y_shuffled, Y_preds = [], []
    for X, Y in loader:
        preds = model(X)
        Y_preds.append(preds)
        Y_shuffled.append(Y)
    gc.collect()
    Y_preds, Y_shuffled = torch.cat(Y_preds), torch.cat(Y_shuffled)

    return Y_shuffled.detach().numpy(), F.softmax(Y_preds, dim=-1).argmax(dim=-1).detach().numpy()



##### Entrenamiento del modelo

Ejecute el entrenamiento de su modelo propuesto.

In [ ]:
from torch.utils.data import DataLoader


rnn_classifier = RNNClassifier(len(vocab), embed_len, hidden_dim, len(target_classes))


train_dataset = load_df(train_df)
test_dataset = load_df(test_df)

train_dataset = to_map_style_dataset(train_dataset)
test_dataset = to_map_style_dataset(test_dataset)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    collate_fn=vectorize_batch,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=vectorize_batch,
)

epochs = 10
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

TrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)




100%|██████████| 306/306 [00:09<00:00, 32.84it/s]


Train Loss : 1.021
Valid Loss : 0.967
Valid Acc  : 0.539


100%|██████████| 306/306 [00:09<00:00, 33.36it/s]


Train Loss : 0.867
Valid Loss : 0.892
Valid Acc  : 0.590


100%|██████████| 306/306 [00:08<00:00, 34.38it/s]


...

100%|██████████| 306/306 [00:08<00:00, 35.51it/s]


Train Loss : 0.129
Valid Loss : 1.254
Valid Acc  : 0.674


100%|██████████| 306/306 [00:08<00:00, 35.59it/s]


Train Loss : 0.088
Valid Loss : 1.347
Valid Acc  : 0.677


'\n\n\n\n\nembed_len = 50\nhidden_dim1 = 50\nhidden_dim2 = 60\nhidden_dim3 = 75\nn_layers=1\n\n\n\n\n\n\n\n\nbatch_size = 32  # o el que uses\n\n\nrnn_classifier = RNNClassifier(len(vocab), embed_len, hidden_dim1, len(target_classes))\nloss_fn = nn.CrossEntropyLoss()\noptimizer = Adam(rnn_classifier.parameters(), lr=learning_rate) \n\n\n#train_loader = DataLoader(train_dataset, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)\n# test_loader  = DataLoader(test_dataset , batch_size=1024, collate_fn=vectorize_batch)\n\nTrainModel(rnn_classifier, loss_fn, optimizer, train_loader, test_loader, epochs)\n'

##### Evaluacion del modelo

Ejecute la evaluación de su modelo, y genere un reporte de evaluación similar al de la pregunta anterior.

In [162]:
Y_actual, Y_preds = MakePredictions(rnn_classifier, test_loader)

In [163]:
print("Test Accuracy : {}".format(accuracy_score(Y_actual, Y_preds)))
print("\nClassification Report : ")
print(classification_report(Y_actual, Y_preds, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual, Y_preds))

Test Accuracy : 0.6827670896438804

Classification Report : 
              precision    recall  f1-score   support

      normal       0.64      0.68      0.66       856
 incivilidad       0.75      0.72      0.74      1085
        odio       0.61      0.59      0.60       502

    accuracy                           0.68      2443
   macro avg       0.67      0.67      0.67      2443
weighted avg       0.68      0.68      0.68      2443


Confusion Matrix : 
[[585 165 106]
 [213 786  86]
 [110  95 297]]


Finalmente, análice sus resultados, ¿Porqué cree que obtuvo esos resultados?, ¿Es mejor que sólo utilizar Word Embeddings, porque?. Justique.

### Comentarios:

El modelo obtuvo un accuracy cercano a 0,68 y un F1 macro de aproximadamente 0,67, lo que indica que es capaz de capturar patrones generales del problema. Sin embargo, la separación entre las categorías “normal”, “incivilidad” y “odio” es por defecto, difusa, especialmente en "incivilidad". En la matriz de confusión, esta es la clase que presenta la mayor cantidad de predicciones erróneas, tanto hacia “normal” como hacia “odio”.

El rendimiento también está condicionado por la calidad de los datos. Si bien se aplicó un preprocesamiento básico, los datos tienen ruido propio del lenguaje de redes sociales. Estos genera dificultades para que un modelo simple pueda distinguir con claridad los matices entre las clases.

En resumen, se obtuvieron resultados apropiados para un modelo basado en RNN con un dataset complejo. Se identifican los patrones de alto nivel, pero los límites entre clases similares semánticamente deterioran los resultados.

Si se utilizaran solo word embeddings, el desempeño sería aún más limitado. Como los embeddings no incorporan contextofrases como “no es odio” y “es odio” quedarían muy próximas en el espacio vectorial, a pesar de transmitir significados opuestos. En cambio, la RNN incorpora secuencia u orden de las palabras y negaciones.



### Transformers BERT.

Para esta tarea se le piden crear una representación de texto usando la arquitectura basada en transformers, BERT:



In [5]:
!pip install transformers


#### Import BETO

Para esto debe importar el tokenizador y el modelo BETO.

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [2]:
import torch

device = torch.device("cpu")


print(torch.__version__)

2.9.1+cpu


#### Ejemplo de extracción de features.

Luego, debe cargar los modelos pre-entrenados:

In [3]:
tokenizer = AutoTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
model = AutoModelForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', output_hidden_states=True)

Una vez cargado BETO, debe utilizarlo para extraer los embeddings asociados a la texto de su corpus. Se espera que usted realice lo siguiente:



Tokenizamos el texto para extraer los ids a cada palabra en el vocabulario interno de BETO, se recomienda utilizar el padding, trunctation, y max_length para considerar oraciones de diferentes tamaños.

Luego, debe verificar si cada uno de los ids extraídos se encuentran en la misma máquina donde fue cargado el modelo (CPU o GPU), se recomienda dejar todo en GPU.

# NOTA : CPU vs GPU

Se realiza la construcción y las pruebas sobre CPU., y se opta por estandarizar en este tipo de arquitectura, por **acceso limitado a GPU**.

In [4]:
# oración
sentence = "Hola, que tal? me gusta mucho el curso de NLP"

# extraemos los ids de los tokens, se recomienda definir los valores de las variables:
#  padding, truncation, max_length debido a que las secuencia de texto puede tener diferentes largos
inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt", max_length=512)
# revisamos si cada de los ids, se encuentran en la misma máquina que el modelo (GPU o CPU)
inputs = {k: v.to(model.device) for k, v in inputs.items()}
inputs

{'input_ids': tensor([[    4,  1734,  1019,  1041,  1713,  1059,  1094,  2331,  1700,  1039,
           4430,  1009, 15567, 30968,     5]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Una vez, extraídos los ids, usted debe obtener los estados ocultos de las ultimas capas de BETO.

In [ ]:
# Extraemos la features
outputs = model(**inputs)

# ahora `outputs` tendrá el atributo `hidden_states`
hidden_states = outputs.hidden_states


(1, 768)

Finalmente, debe guardar los embeddings en CPU los embeddings del token [CLS] que será usados en la clasificación del análisis de sentimientos.

In [6]:
with torch.no_grad():
# Seleccionamos la última capa y obtenemos el primer token ([CLS]) para cada ejemplo
# Estos son los embeddings que normalmente se usan para tareas de clasificación.
  cls_embeddings = hidden_states[-1][:, 0, :].cpu().numpy()


#### Extraer features de todo el dataset

Considerando lo anterior, usted debe implementar la función `get_beto_features_in_batches` para extraer los features de BETO (los estados ocultos y los embeddings del token [CLS]).

Esta función recibe dos parámetros, el texto a vectorizar y un tamaño de batch, debido a que es extramadamente recomendable a que procesen el texto por lotes, ya que si cargan todos el modelo se les agotará la memoria RAM.

In [10]:
# Función para procesar los textos en lotes y obtener las características de BETO

def get_beto_features_in_batches(texts, batch_size=32, max_length=128):
    all_cls_embeddings = []

    # Seteo en modo evaluación
    model.eval()

    for start in range(0, len(texts), batch_size):
        end = start + batch_size
        batch_texts = list(texts[start:end])

        # Tokenización con padding/truncation
        batch_inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )

        # Enviar a mismo dispositivo del modelo (CPU o GPU)
        batch_inputs = {k: v.to(model.device) for k, v in batch_inputs.items()}

        # Forward sin gradientes
        with torch.no_grad():
            outputs = model(**batch_inputs)
            hidden_states = outputs.hidden_states
            last_hidden = hidden_states[-1]          # (batch, seq_len, hidden_size)
            cls_batch = last_hidden[:, 0, :]         # (batch, hidden_size)

        # Guardar embeddings en CPU
        all_cls_embeddings.append(cls_batch.cpu().numpy())

        # Liberar GPU un poco si corresponde
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Concatenamos todos los batches
    all_cls_embeddings = np.vstack(all_cls_embeddings)
    return all_cls_embeddings

ejemplos = ["Me encanta este curso", "No me gusta este comentario"]
embs_ejemplo = get_beto_features_in_batches(ejemplos, batch_size=2)
embs_ejemplo.shape  # (2, 768) normalmente

NameError: name 'np' is not defined

Ahora extraíga los features, un punto importante es que la extracción de features podría tomar alrededor de una a dos horas dependiendo del tamaño del batch que utilicen.

In [ ]:
train_embs = get_beto_features_in_batches(..., ...)

#### Clasificación

Una vez extraído los features de BETO, realice la clasificación de los embeddings obtenidos. Debe implementar dos clasificadores a su elección.

#### Reporte de evaluación

Una vez realizada la clasificación, realice el reporte de clasificación y el análsis de la matriz confusión para ambos clasificadores.

Recuerde que para hacer esto, debe extraer los features del dataset de testing.

Finalmente, que puede decir de los resultados obtenidos. ¿Se diferencia de los resultados obtenidos de la red RNN? ¿A que se debe esto? Justifique

### Large Language Model

##### Zero Shot Learning

Utilizando la técnica de zero shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.

##### Few Shot Learning

Utilizando la técnica de few shot learning, utilice la API de `openai` para clasificar el texto del dataset.

Además, genere el reporte de clasificación usando `scikit-learn` como en las preguntas anteriores.

Recuerde solicitar al profesor auxiliar el TOKEN para hacer consultas al LLM.